In [3]:
import os
import requests
import tarfile

class movieDataAnalyzer:
    def __init__(self, data_url):
        """
        Initialize the movieDataAnalyzer class.
        
        Args:
            data_url (str): URL of the data file to download
        """
        # Set the download directory
        self.download_dir = '../downloads'

        # Set the file path
        filename = 'MovieSummaries' 
        self.file_path = os.path.join(self.download_dir, filename)
        
        # Create the download directory if it doesn't exist
        if not os.path.exists(self.download_dir):
            os.makedirs(self.download_dir)
        
        # Download the file if it doesn't exist
        if not os.path.exists(self.file_path):
            print(f"Downloading Data...")
            self._download_file(data_url)
        else:
            print(f"File {filename} already exists. Skipping download.")
    
    def _download_file(self, url):
        """Download and extract the file from the provided URL."""
        # Download the file
        response = requests.get(url, stream=True)
        response.raise_for_status()
        
        with open(self.file_path, 'wb') as f:
            for chunk in response.iter_content(chunk_size=8192):
                if chunk:
                    f.write(chunk)
        print("Download complete.")
        
        # Extract the tarball
        print(f"Extracting {os.path.basename(self.file_path)}...")
        with tarfile.open(self.file_path, 'r:gz') as tar:
            tar.extractall(path=self.download_dir)
        print("Extraction complete.")

# Download the data file
data_url = 'http://www.cs.cmu.edu/~ark/personas/data/MovieSummaries.tar.gz'
filename = 'MovieSummaries.tar.gz'

# Initialize the movieDataAnalyzer class
mda = movieDataAnalyzer(data_url)


File MovieSummaries already exists. Skipping download.
